In [123]:
from itertools import product
from functools import partial
import os
from typing import Any, Dict, Callable, List, Tuple
import zipfile

import gensim.models
from navec import Navec
import numpy as np
import pandas as pd
import pymorphy3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
import tqdm
import urllib.request

In [2]:
SEED = 42

In [3]:
def split(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    global SEED
    train_val_df, test_df = train_test_split(
        df, test_size=0.2, random_state=SEED, stratify=df["label"].values
    )
    train_df, val_df = train_test_split(
        train_val_df,
        test_size=0.2 * df.shape[0] / train_val_df.shape[0],
        random_state=SEED,
        stratify=train_val_df["label"].values,
    )
    return train_df, val_df, test_df


def tokenize(text: str) -> List[str]:
    return text.split(" ")

In [4]:
tokens_df = pd.read_csv("../data/hw1/processed_df.csv")
train_df, val_df, test_df = split(tokens_df)

In [5]:
train_df

,filtered_text,label
69234,экономика великобритании году сократится проце...,Экономика
36160,в минобороны россии назвали эффективность амер...,Силовые структуры
7065,американский боксер мохаммед али госпитализиро...,Спорт
53795,на окраине мурманска вторник обнаружен подполь...,Россия
63444,пользователь яндекса требует взыскать поискови...,Интернет и СМИ
...,...,...
45054,до конца года будут проведены учебно-боевых пу...,Силовые структуры
49047,комиссия российской академии наук изучив прете...,Россия
87334,компания ikea выпустила набор рецептов котором...,Интернет и СМИ
44320,в одном мурманских скверов июня года откроют с...,69-я параллель


## Обучение собственных эмбеддингов word2vec:
Обоснуем выбор некоторых гиперпараметров.

- vector_size - есть внутреннее ощущение, что слишком большие эмбеддинги могут переобучиться на этой задаче, поэтому попробуем начать с 250
- window - в задаче классификации текстов нам важно получать контекстную близость, а не синонимическую
- min_count - упрощаем обучение уменьшая маловстречаемые слова
- sg - Skip Gramm, чтобы быстрее учиться
- negative - аналогично, на каждой итерации вместо софтмакса по всему датасету считаем только 5 слов
- epochs - чем больше датасет, тем больше эпох можем поставить (обычно 10-50), начнем с 30


In [6]:
train_data = train_df["filtered_text"].apply(tokenize).values.tolist()

In [7]:
if os.path.exists("w2v_250_w10_mc5_sg_neg5_ep30"):
    model = gensim.models.Word2Vec.load("w2v_250_w10_mc5_sg_neg5_ep30")
else:
    model = gensim.models.Word2Vec(
        sentences=train_data,
        vector_size=250,
        window=10,
        min_count=5,
        sg=1,
        negative=5,
        epochs=30,
        seed=SEED,
    )
    model.save("w2v_250_w10_mc5_sg_neg5_ep30")

In [8]:
print(" | ".join(train_df["label"].unique().tolist()))

Экономика | Силовые структуры | Спорт | Россия | Интернет и СМИ | Из жизни | Мир | Наука и техника | Дом | Бывший СССР | Культура | Ценности | Путешествия | Бизнес | Легпром | 69-я параллель | Крым | Культпросвет  | Библиотека


Попробуем оценить качество эмбеддингов, используя самые частотные

In [9]:
model.wv.most_similar("экономика")

[('ввп', 0.6906951665878296),
 ('рецессии', 0.6488956212997437),
 ('рецессию', 0.6266115307807922),
 ('замедлятся', 0.6121088862419128),
 ('адаптировалась', 0.5717689394950867),
 ('экономики', 0.5682619214057922),
 ('инфляция', 0.5424972176551819),
 ('экономический', 0.5400095582008362),
 ('рецессия', 0.5394936203956604),
 ('рецессией', 0.5338839292526245)]

In [10]:
model.wv.most_similar(negative=["экономика"])

[('нигерийского', 0.1339004635810852),
 ('прицельный', 0.12875449657440186),
 ('рыжего', 0.11170051246881485),
 ('палестинца', 0.11158581078052521),
 ('стрельбой', 0.10589572787284851),
 ('четверым', 0.10377943515777588),
 ('гаражного', 0.10017924010753632),
 ('пятерым', 0.09932998567819595),
 ('самары', 0.09752027690410614),
 ('красноярска', 0.09594501554965973)]

In [11]:
model.wv.most_similar("мвд")

[('гу', 0.7301331162452698),
 ('милиции', 0.6089017391204834),
 ('следственного', 0.6055797934532166),
 ('главка', 0.6005496382713318),
 ('внутренних', 0.5966971516609192),
 ('увд', 0.5913692712783813),
 ('оперативно-розыскного', 0.5893971920013428),
 ('говд', 0.5781435966491699),
 ('гувд', 0.5723111629486084),
 ('фсб', 0.5708879232406616)]

In [12]:
model.wv.most_similar(negative=["мвд"])

[('ямайка', 0.1073177233338356),
 ('дилан', 0.10620534420013428),
 ('melissa', 0.10451147705316544),
 ('консультировал', 0.10422379523515701),
 ('здоровой', 0.10061188787221909),
 ('подниматься', 0.1005881130695343),
 ('фазе', 0.09912239015102386),
 ('высказывают', 0.095782071352005),
 ('lake', 0.09563589841127396),
 ('охлаждении', 0.09526385366916656)]

In [13]:
model.wv.most_similar("футбол")

[('баскетбол', 0.5135613679885864),
 ('футболистов', 0.4858241677284241),
 ('атлетику', 0.48419076204299927),
 ('матч', 0.47358715534210205),
 ('играть', 0.47240549325942993),
 ('легии', 0.4598545730113983),
 ('футбольный', 0.45508161187171936),
 ('болельщики', 0.4550151824951172),
 ('расслабляться', 0.4545561671257019),
 ('футбола', 0.45173168182373047)]

In [14]:
model.wv.most_similar(negative=["футбол"])

[('фгупов', 0.12461049109697342),
 ('сертификации', 0.12004587799310684),
 ('подлежат', 0.1166456788778305),
 ('вывезены', 0.11573202162981033),
 ('имелся', 0.11476853489875793),
 ('согласованию', 0.11053688079118729),
 ('печатью', 0.11017366498708725),
 ('сложностях', 0.10659559071063995),
 ('отозвало', 0.10580680519342422),
 ('позволяло', 0.10374383628368378)]

Выглядит так, что ключевые слова разделены по категориям, поэтому параметры подобраны достаточно хорошо

In [17]:
urllib.request.urlretrieve(
    "https://vectors.nlpl.eu/repository/20/184.zip", "news_upos_skipgram_300_5_2019.zip"
)
zip_path = "news_upos_skipgram_300_5_2019.zip"
extract_folder = "news_upos_skipgram_300_5_2019"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_folder)

In [15]:
rusvec_model_path = "news_upos_skipgram_300_5_2019/model.bin"
rusvec_model = gensim.models.KeyedVectors.load_word2vec_format(
    rusvec_model_path, binary=True
)

In [19]:
rusvec_model.most_similar("мвд_NOUN")

[('гумвд_NOUN', 0.4122755229473114),
 ('уфсб_NOUN', 0.41181737184524536),
 ('гувд_NOUN', 0.392577588558197),
 ('адриан::ранкин-гэллоуэй_PROPN', 0.3685126304626465),
 ('алиевой_NOUN', 0.3653308153152466),
 ('возражалийский_ADJ', 0.3638455271720886),
 ('мчс::россия::раиса::копырина_PROPN', 0.3625434935092926),
 ('амра::юзбеков_PROPN', 0.36013513803482056),
 ('республиканца_NOUN', 0.3595658838748932),
 ('мвд::россия_PROPN', 0.35692664980888367)]

И теперь из navec

In [22]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
path = "navec_hudlit_v1_12B_500K_300d_100q.tar"
navec_model = Navec.load(path)

In [17]:
def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0
    return np.dot(vec1, vec2) / (norm1 * norm2)


def ineffective_most_similiar(
    navec_model: Navec, word: str, top_n: int = 10
) -> List[Tuple[str, float]]:
    if word not in navec_model:
        raise ValueError(f"Слова {word} нет в модели")

    word_vector = navec_model[word]
    similarities = {}
    for other_word in navec_model.vocab.words:
        other_vector = navec_model[other_word]
        other_norm = np.linalg.norm(other_vector)

        if other_norm == 0:
            continue

        similarities[other_word] = cosine_similarity(word_vector, other_vector)

    return sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_n]

In [26]:
ineffective_most_similiar(navec_model, "мвд")

[('мвд', 1.0),
 ('фсб', 0.76623964),
 ('кгб', 0.7264482),
 ('мгб', 0.69483584),
 ('гувд', 0.68139136),
 ('нквд', 0.6729191),
 ('госбезопасности', 0.6632358),
 ('увд', 0.6568187),
 ('гру', 0.64401966),
 ('прокуратуры', 0.6382184)]

In [18]:
vector_size = navec_model.pq.dim
gensim_from_navec = gensim.models.KeyedVectors(vector_size)
words, vectors = [], []
for word in navec_model.vocab.words:
    vec = navec_model[word]
    if np.linalg.norm(vec) > 0:
        words.append(word)
        vectors.append(vec)

gensim_from_navec.add_vectors(words, np.array(vectors))

Есть ощущение, что вектора из navec и rusvector больше склонны показывать функциональную близость - т.е. слова, которые являются почти синонимами

Обучим на каждом виде эмбеддингов модель логистической регрессии

In [19]:
def create_features(
    data: pd.DataFrame,
    emb_model: gensim.models.Word2Vec,
    use_tag: bool = False,
    is_kv: bool = False,
    emb_dim: int = 300,
) -> np.ndarray:
    if use_tag:
        morph = pymorphy3.MorphAnalyzer()
    X = np.empty((0, emb_dim))
    for text in data["filtered_text"].values:
        if use_tag:
            pos_tags = [morph.parse(word)[0].tag.POS for word in text.split(" ")]
        avg_emb = np.empty((0, emb_dim))
        for i, token in enumerate(text.split(" ")):
            token = f"{token}_{pos_tags[i]}" if use_tag else token
            embs_storage = emb_model if is_kv else emb_model.wv
            if token in embs_storage:
                emb = embs_storage[token]
                avg_emb = np.vstack((avg_emb, emb))
        if len(avg_emb) > 0:
            avg_emb = np.mean(avg_emb, axis=0).reshape(1, emb_dim)
        else:
            avg_emb = np.zeros((1, emb_dim))
        X = np.vstack((X, avg_emb))
    return X


In [ ]:
custom_train_features = create_features(train_df, model, emb_dim=250)
rusvec_train_features = create_features(
    train_df, rusvec_model, use_tag=True, is_kv=True, emb_dim=300
)
navec_train_features = create_features(
    train_df, gensim_from_navec, is_kv=True, emb_dim=300
)

In [211]:
for fpath, features in zip(
    ["custom_train", "rusvec_train", "navec_train"],
    [custom_train_features, rusvec_train_features, navec_train_features],
):
    with open(f"../data/hw2/{fpath}.npy", "wb") as f:
        np.save(f, features)

In [20]:
with open("../data/hw2/custom_train.npy", "rb") as f:
    custom_train_features = np.load(f)
with open("../data/hw2/rusvec_train.npy", "rb") as f:
    rusvec_train_features = np.load(f)
with open("../data/hw2/navec_train.npy", "rb") as f:
    navec_train_features = np.load(f)

In [31]:
custom_val_features = create_features(val_df, model, emb_dim=250)
rusvec_val_features = create_features(
    val_df, rusvec_model, use_tag=True, is_kv=True, emb_dim=300
)
navec_val_features = create_features(val_df, gensim_from_navec, is_kv=True, emb_dim=300)

In [32]:
for fpath, features in zip(
    ["custom_val", "rusvec_val", "navec_val"],
    [custom_val_features, rusvec_val_features, navec_val_features],
):
    with open(f"../data/hw2/{fpath}.npy", "wb") as f:
        np.save(f, features)

In [21]:
with open("../data/hw2/custom_val.npy", "rb") as f:
    custom_val_features = np.load(f)
with open("../data/hw2/rusvec_val.npy", "rb") as f:
    rusvec_val_features = np.load(f)
with open("../data/hw2/navec_val.npy", "rb") as f:
    navec_val_features = np.load(f)

In [34]:
custom_test_features = create_features(test_df, model, emb_dim=250)
rusvec_test_features = create_features(
    test_df, rusvec_model, use_tag=True, is_kv=True, emb_dim=300
)
navec_test_features = create_features(
    test_df, gensim_from_navec, is_kv=True, emb_dim=300
)

In [35]:
for fpath, features in zip(
    ["custom_test", "rusvec_test", "navec_test"],
    [custom_test_features, rusvec_test_features, navec_test_features],
):
    with open(f"../data/hw2/{fpath}.npy", "wb") as f:
        np.save(f, features)

In [22]:
with open("../data/hw2/custom_test.npy", "rb") as f:
    custom_test_features = np.load(f)
with open("../data/hw2/rusvec_test.npy", "rb") as f:
    rusvec_test_features = np.load(f)
with open("../data/hw2/navec_test.npy", "rb") as f:
    navec_test_features = np.load(f)

In [23]:
def train_logreg(X: np.ndarray, y: np.ndarray) -> LogisticRegression:
    logreg = LogisticRegression(max_iter=2000)
    logreg.fit(X, y)
    return logreg


y = train_df["label"].values
custom_fs_logreg = train_logreg(custom_train_features, y)
rusvec_fs_logreg = train_logreg(rusvec_train_features, y)
navec_fs_logreg = train_logreg(navec_train_features, y)

In [38]:
custom_vp = custom_fs_logreg.predict(custom_val_features)
print(classification_report(val_df["label"].values, custom_vp, zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       1.00      0.06      0.11        35
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.65      0.15      0.25       200
      Бывший СССР       0.82      0.76      0.79      1444
              Дом       0.84      0.74      0.79       588
         Из жизни       0.64      0.59      0.61       747
   Интернет и СМИ       0.76      0.68      0.72      1209
             Крым       0.00      0.00      0.00        18
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.87      0.86      0.87      1455
          Легпром       0.00      0.00      0.00         3
              Мир       0.77      0.85      0.81      3697
  Наука и техника       0.82      0.81      0.82      1434
      Путешествия       0.74      0.45      0.56       174
           Россия       0.75      0.84      0.79      4338
Силовые структуры       0.65      0.28      0.39       

In [39]:
rusvec_vp = rusvec_fs_logreg.predict(rusvec_val_features)
print(classification_report(val_df["label"].values, rusvec_vp, zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       0.45      0.14      0.22        35
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.42      0.10      0.16       200
      Бывший СССР       0.71      0.61      0.65      1444
              Дом       0.73      0.69      0.71       588
         Из жизни       0.55      0.46      0.50       747
   Интернет и СМИ       0.63      0.52      0.57      1209
             Крым       0.00      0.00      0.00        18
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.81      0.81      0.81      1455
          Легпром       0.00      0.00      0.00         3
              Мир       0.70      0.78      0.74      3697
  Наука и техника       0.72      0.74      0.73      1434
      Путешествия       0.55      0.36      0.43       174
           Россия       0.69      0.78      0.73      4338
Силовые структуры       0.49      0.20      0.28       

In [40]:
navec_vp = navec_fs_logreg.predict(navec_val_features)
print(classification_report(val_df["label"].values, navec_vp, zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       0.50      0.03      0.05        35
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.64      0.12      0.19       200
      Бывший СССР       0.75      0.70      0.73      1444
              Дом       0.79      0.72      0.75       588
         Из жизни       0.62      0.56      0.59       747
   Интернет и СМИ       0.71      0.64      0.67      1209
             Крым       0.00      0.00      0.00        18
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.85      0.85      0.85      1455
          Легпром       0.00      0.00      0.00         3
              Мир       0.76      0.83      0.79      3697
  Наука и техника       0.80      0.80      0.80      1434
      Путешествия       0.63      0.43      0.51       174
           Россия       0.73      0.81      0.77      4338
Силовые структуры       0.54      0.23      0.32       

Кастомные эмбедиднги показали лучший результат по всем средним метрикам

Попробуем применить веса tf-idf для эмбеддингов

In [26]:
tf_idf_param_grid = {"min_df": [500, 1000, 5000], "max_df": [0.8, 0.9, 0.95]}
param_combs = list(product(tf_idf_param_grid["min_df"], tf_idf_param_grid["max_df"]))
param_combs = [{"min_df": comb[0], "max_df": comb[1]} for comb in param_combs]

In [148]:
def get_weighted_features(
    weights: np.ndarray,
    emb_size: int,
    texts: np.ndarray,
    model: gensim.models.Word2Vec,
    tf_idf_vectorizer: TfidfVectorizer,
) -> np.ndarray:
    vocabulary = tf_idf_vectorizer.vocabulary_
    embeddings = np.zeros((len(texts), emb_size))

    for i, text in enumerate(texts):
        tokens = text.split()
        token_weights = []
        token_embs = []

        for token in tokens:
            word_id = vocabulary.get(token)
            if word_id is None or weights[i, word_id] == 0:
                continue
            try:
                emb = model.wv.get_vector(token)
            except KeyError:
                continue

            token_weights.append(weights[i, word_id])
            token_embs.append(emb * weights[i, word_id])

        if token_embs:
            embeddings[i] = np.average(token_embs, axis=0, weights=token_weights)

    return embeddings


def find_hyperparams(
    model: gensim.models.Word2Vec,
    param_grid: Dict[str, Any],
    emb_size: int = 250,
    metric_funk: Callable = f1_score,
    more_is_better: bool = True,
) -> Dict[str, Any]:
    param_combs = list(product(*param_grid.values()))
    param_combs = [
        {key: val for key, val in zip(param_grid.keys(), comb)} for comb in param_combs
    ]
    results = []
    train_y = train_df["label"].values
    val_y = val_df["label"].values
    for params in tqdm.tqdm(param_combs):
        tf_idf_vectorizer = TfidfVectorizer(**params)
        train_tfidfw = tf_idf_vectorizer.fit_transform(train_df["filtered_text"].values)
        val_tfidfw = tf_idf_vectorizer.transform(val_df["filtered_text"].values)
        train_X = get_weighted_features(
            train_tfidfw,
            emb_size,
            train_df["filtered_text"].values,
            model,
            tf_idf_vectorizer,
        )
        val_X = get_weighted_features(
            val_tfidfw,
            emb_size,
            val_df["filtered_text"].values,
            model,
            tf_idf_vectorizer,
        )
        logreg = LogisticRegression(max_iter=3500)
        logreg.fit(train_X, train_y)
        val_preds = logreg.predict(val_X)
        criteria = metric_funk(val_y, val_preds)
        results.append({**params, "criteria": criteria})
    best_params = list(
        sorted(results, key=lambda x: x["criteria"], reverse=more_is_better)
    )[0]
    return best_params

In [149]:
best_params = find_hyperparams(
    model, tf_idf_param_grid, 250, partial(f1_score, average="macro")
)

100%|██████████| 9/9 [21:45<00:00, 145.10s/it]


In [151]:
train_val_data = (
    train_df["filtered_text"].values.tolist() + val_df["filtered_text"].values.tolist()
)
train_val_y = train_df["label"].values.tolist() + val_df["label"].values.tolist()

In [152]:
tf_idf_params = {**best_params}
del tf_idf_params["criteria"]

In [153]:
tf_idf_vect = TfidfVectorizer(**tf_idf_params)
train_val_weights = tf_idf_vect.fit_transform(train_val_data)
train_val_features = get_weighted_features(
    train_val_weights, 250, train_val_data, model, tf_idf_vect
)
ctv_w2v_ifidf_logreg = LogisticRegression(max_iter=3000)
ctv_w2v_ifidf_logreg.fit(train_val_features, train_val_y)

LogisticRegression(max_iter=3000)

In [154]:
test_weights = tf_idf_vect.transform(test_df["filtered_text"].values)
test_features = get_weighted_features(
    test_weights,
    250,
    test_df["filtered_text"].values,
    model,
    tf_idf_vect
)

In [155]:
test_y = test_df["label"].values
test_preds = ctv_w2v_ifidf_logreg.predict(test_features)
print(classification_report(test_y, test_preds, zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00        34
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.17      0.01      0.01       200
      Бывший СССР       0.73      0.44      0.55      1444
              Дом       0.78      0.55      0.64       588
         Из жизни       0.47      0.22      0.30       747
   Интернет и СМИ       0.63      0.44      0.52      1208
             Крым       0.00      0.00      0.00        18
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.72      0.70      0.71      1456
          Легпром       0.00      0.00      0.00         3
              Мир       0.61      0.78      0.68      3697
  Наука и техника       0.68      0.69      0.69      1435
      Путешествия       0.71      0.06      0.11       173
           Россия       0.59      0.77      0.67      4338
Силовые структуры       0.48      0.04      0.08       

Сравним с термя моделями, которые были получены ранее

In [156]:
custom_tp = custom_fs_logreg.predict(custom_test_features)
print(classification_report(test_y, custom_tp, zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       0.86      0.18      0.29        34
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.59      0.12      0.19       200
      Бывший СССР       0.81      0.79      0.80      1444
              Дом       0.87      0.75      0.81       588
         Из жизни       0.61      0.51      0.56       747
   Интернет и СМИ       0.73      0.67      0.70      1208
             Крым       0.00      0.00      0.00        18
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.86      0.87      0.86      1456
          Легпром       0.00      0.00      0.00         3
              Мир       0.77      0.84      0.81      3697
  Наука и техника       0.83      0.81      0.82      1435
      Путешествия       0.76      0.56      0.65       173
           Россия       0.74      0.83      0.78      4338
Силовые структуры       0.65      0.26      0.37       

In [157]:
rusvec_tp = rusvec_fs_logreg.predict(rusvec_test_features)
print(classification_report(test_y, rusvec_tp, zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       0.71      0.29      0.42        34
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.45      0.09      0.15       200
      Бывший СССР       0.72      0.64      0.68      1444
              Дом       0.76      0.70      0.73       588
         Из жизни       0.51      0.40      0.45       747
   Интернет и СМИ       0.61      0.54      0.57      1208
             Крым       0.33      0.11      0.17        18
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.79      0.81      0.80      1456
          Легпром       0.00      0.00      0.00         3
              Мир       0.71      0.78      0.74      3697
  Наука и техника       0.73      0.74      0.74      1435
      Путешествия       0.58      0.35      0.43       173
           Россия       0.69      0.78      0.73      4338
Силовые структуры       0.43      0.18      0.25       

In [158]:
navec_tp = navec_fs_logreg.predict(navec_test_features)
print(classification_report(test_y, navec_tp, zero_division=0))

                   precision    recall  f1-score   support

   69-я параллель       0.86      0.18      0.29        34
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.41      0.10      0.15       200
      Бывший СССР       0.76      0.70      0.73      1444
              Дом       0.83      0.76      0.79       588
         Из жизни       0.58      0.51      0.54       747
   Интернет и СМИ       0.67      0.62      0.65      1208
             Крым       0.00      0.00      0.00        18
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.84      0.86      0.85      1456
          Легпром       0.00      0.00      0.00         3
              Мир       0.76      0.81      0.78      3697
  Наука и техника       0.78      0.80      0.79      1435
      Путешествия       0.61      0.50      0.55       173
           Россия       0.72      0.80      0.76      4338
Силовые структуры       0.53      0.20      0.29       

Интересно, что в итоге наилучшее качество было получено через обычное усреднение без учета tf-idf..

Но это хотя бо соотносится с результатами первой ДЗ, где tf-idf показал себя хуже, чем count_vectorizer)